### Setting up Training Data ###

In [1]:
import numpy as np
import h5py
import csv
import matplotlib.pyplot as plt
with h5py.File("data.hdf5", 'r') as f:
    train_data = f['train'][()]
    test_data = f['test'][()]
labels = open('train_labels.csv')
labels_reader = csv.reader(labels)
classes = ["FM","OQPSK","BPSK","8PSK","AM-SSB-SC","4ASK","16PSK","AM-DSB-SC","QPSK","OOK"]
train_labels_data = []
for row in labels_reader:
    if row[0] != 'Id':
        train_labels_data.append(classes.index(row[1]))

train_labels_data = np.asarray(train_labels_data)
print(train_data.shape)
print(len(train_labels_data))
print(test_data.shape)

    


(30000, 1024, 2)
30000
(20000, 1024, 2)


### Visualizing Data  ###

In [2]:
# for j in range(0,50):
#     I = []
#     Q = []
#     for i in range(0,1024):
#         I.append(train_data[j][i][0])
#         Q.append(train_data[j][i][1])
#     plt.figure(j)
#     plt.scatter(I,Q)

# fft_train_2d.shape

### Setting up Convolutional Neural Network Architecture ###

In [9]:
from __future__ import print_function
import torch
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn


# converting training images into torch format
train_x = train_data.reshape(30000, 1, 1024, 2)
train_x  = torch.from_numpy(train_x)

# converting test images into torch format
test_x = test_data.reshape(20000, 1, 1024, 2)
test_x  = torch.from_numpy(test_x)

# converting the target into torch format
train_y = train_labels_data.astype(int)
train_y = torch.from_numpy(train_y)

# split into batches
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.6,shuffle=True)


train_x1, train_x2, train_y1,train_y2 = train_test_split(train_x, train_y, test_size = 0.50,shuffle=True)
train_x3, train_x4, train_y3,train_y4 = train_test_split(train_x1, train_y1, test_size = 0.50,shuffle=True)
train_x5, train_x6, train_y5,train_y6 = train_test_split(train_x2, train_y2, test_size = 0.7,shuffle=True)
# batches

# shape of training data
print(train_x.shape)
print(train_y.shape)



class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 5, kernel_size=(3,3), stride=1, padding=7)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))
        self.conv2 = nn.Conv2d(5, 5, kernel_size=5, stride=1, padding=2)
        self.dropout = nn.Dropout(p=0.06)
        self.batchnorm = nn.BatchNorm2d(num_features=5)
        self.fc1 = nn.Linear(3885, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, len(classes))

    def forward(self, x):
        x = self.pool(F.selu(self.batchnorm(self.conv1(x))))
        x = self.pool(F.selu((self.conv2(x))))
        x = x.view(-1, 3885)
        x = F.selu(self.dropout(self.fc1(x)))
        x = F.selu((self.fc2(x)))
        x = self.fc3(x)
        return x
    
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 128, kernel_size=(5,1), stride=1)
        self.conv2 = nn.Conv2d(128, 40, kernel_size=(5,2), stride=1)
        self.conv3 = nn.Conv2d(40, 40, kernel_size=(5,1), stride=1)
        self.fc1 = nn.Linear(40320, 128)
        self.fc2 = nn.Linear(128, len(classes))

    def forward(self, x):
        x = F.selu(self.conv1(x))
        x = F.selu(self.conv2(x))
        x = F.selu(self.conv3(x))
        x = F.selu(self.conv3(x))
        x = x.view(-1, 40320)
        x = F.selu(self.fc1(x))
        x = F.selu(self.fc2(x))
        
# defining the model
model = Net1()
mode = model.float()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0005,weight_decay=0.001)
# defining the loss function
criterion = CrossEntropyLoss()
print(model)    


#calculate dimensions
x = torch.randn(3,1,1024,2)
conv1 = nn.Conv2d(1, 5, kernel_size=(3,3), stride=1, padding=7)
y=conv1(x)
print(y.shape)
pool = nn.MaxPool2d(kernel_size=(2,2))
y=pool(y)
print(y.shape)
conv2 = nn.Conv2d(5, 5, kernel_size=5, stride=1, padding=2)
y=conv2(y)
print(y.shape)
y=pool(y)
print(y.shape)

torch.Size([12000, 1, 1024, 2])
torch.Size([12000])
Net1(
  (conv1): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(7, 7))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(5, 5, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (dropout): Dropout(p=0.06, inplace=False)
  (batchnorm): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3885, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)
torch.Size([3, 5, 1036, 14])
torch.Size([3, 5, 518, 7])
torch.Size([3, 5, 518, 7])
torch.Size([3, 5, 259, 3])


# Set up training protocol ###

In [10]:
def train(epoch,train_x,train_y):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    
    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training 
    output_train = model(x_train.float())

    # computing the training loss
    loss_train = criterion(output_train, y_train)
    train_losses.append(loss_train)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    
    with torch.no_grad():
        output = model(train_x)
    
    softmax = torch.exp(output)
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    print(accuracy_score(train_y, predictions))
    
    train_acc.append(accuracy_score(train_y, predictions))
    # printing the training loss
    print('Epoch : ',epoch+1, '\t', 'loss :', loss_train)

In [11]:
# defining the number of epochs
n_epochs = 10000
# empty list to store training losses
train_losses = []
train_acc = []

for epoch in range(n_epochs):
    train(epoch,train_x,train_y)
 
plt.plot(train_losses, label='Training loss')
plt.legend()
plt.show()

0.10183333333333333
Epoch :  1 	 loss : tensor(2.3446, grad_fn=<NllLossBackward>)
0.108
Epoch :  2 	 loss : tensor(2.9221, grad_fn=<NllLossBackward>)
0.11133333333333334
Epoch :  3 	 loss : tensor(2.7062, grad_fn=<NllLossBackward>)
0.10108333333333333
Epoch :  4 	 loss : tensor(2.4701, grad_fn=<NllLossBackward>)
0.10133333333333333
Epoch :  5 	 loss : tensor(2.3880, grad_fn=<NllLossBackward>)
0.10458333333333333
Epoch :  6 	 loss : tensor(2.3721, grad_fn=<NllLossBackward>)
0.10166666666666667
Epoch :  7 	 loss : tensor(2.3624, grad_fn=<NllLossBackward>)
0.10216666666666667
Epoch :  8 	 loss : tensor(2.3510, grad_fn=<NllLossBackward>)
0.10116666666666667
Epoch :  9 	 loss : tensor(2.3372, grad_fn=<NllLossBackward>)
0.10975
Epoch :  10 	 loss : tensor(2.3209, grad_fn=<NllLossBackward>)
0.13158333333333333
Epoch :  11 	 loss : tensor(2.3034, grad_fn=<NllLossBackward>)
0.15333333333333332
Epoch :  12 	 loss : tensor(2.2888, grad_fn=<NllLossBackward>)
0.16041666666666668
Epoch :  13 	 loss 

KeyboardInterrupt: 

In [ ]:



maxprob = [1]*len(train_acc)
error = []

zip_object = zip(maxprob, train_acc)
for i, j in zip_object:
    error.append(i-j)

plt.figure(0)
plt.plot(error, label='Training error')
plt.legend()
plt.show()


plt.figure(1)
plt.plot(train_losses, label='Training loss')
plt.legend()
plt.show()

### Predictions ###

In [12]:
with torch.no_grad():
        output = model(test_x)
softmax = torch.exp(output)
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)


In [13]:
pred_label = []
for i in predictions:
    pred_label.append(classes[i])
    
    
    
    
    
    

In [ ]:
print(pred_label)




In [14]:
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
import pandas as pd
# define id
id = np.arange(0,20000)
labels=np.asarray(pred_label)
labels.reshape(20000,1)
print(labels.shape)
d = {'Category':labels}
df = pd.DataFrame(data=d)




(20000,)


In [15]:
df.to_csv('submission.csv')


In [ ]:
with torch.no_grad():
        output = model(val_x)
    
softmax = torch.exp(output)
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)
print(accuracy_score(val_y, predictions))